In [ ]:
import sys
import argparse
import numpy as np
import scipy.signal as sp
import processing as pr
import matplotlib.pyplot as plt
from ruamel.yaml import YAML as ym

sys.path.append("../preprocessing")
from generate_chirp import generate_chirp

# Widgets are only needed if you want to use the interactive plot at the end
# Installation instructions: https://ipywidgets.readthedocs.io/en/latest/user_install.html
#import ipywidgets as widgets

In [ ]:
# == Use this to plot the last recorded data ==
# yaml_file = "../config/default.yaml"
# bin_file = "../data/rx_samps.bin"

# == Or this to plot saved data ==
prefix = "../data/20211110_142243"
yaml_file = prefix + "_config.yaml"
bin_file = prefix + "_rx_samps.bin"

n_stack = 1 # Do additional stacking in this notebook - set > 1 to enable

In [ ]:
# Initialize Constants
yaml = ym()
with open(yaml_file) as stream:
    config = yaml.load(stream)
    rx_params = config["PLOT"]
    sample_rate = rx_params["sample_rate"]    # Hertz
    rx_samps = bin_file # TODO
    orig_ch = rx_params["orig_chirp"]         # Chirp associated with the received data
    direct_start = rx_params["direct_start"]
    echo_start = rx_params["echo_start"]
    sig_speed = rx_params["sig_speed"]

    expected_n_rxs = int(config['CHIRP']['num_pulses'] / config['CHIRP']['num_presums'])
    
    rx_len_samples = int(config['CHIRP']['rx_duration'] * config['GENERATE']['sample_rate'])

In [ ]:
# Read and plot RX/TX
# This cell loads all of the data - it can take a while with a large file. You don't need to re-run this cell if you only change n_stack
_, tx_sig = generate_chirp(config)
pr.plotChirpVsTime(tx_sig, 'Transmitted Chirp', sample_rate)
print(f"len(tx_sig): {len(tx_sig)}")

rx_sig = pr.extractSig(rx_samps)

In [ ]:
# Coherent stacking

n_rxs = len(rx_sig) // rx_len_samples
rx_sig_reshaped = np.transpose(np.reshape(rx_sig, (n_rxs, rx_len_samples), order='C'))
print(f"len(rx_sig): {len(rx_sig)}")
print(f"n_rxs: {n_rxs}")
print(f"rx_sig shape: {np.shape(rx_sig)}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")

xcorr_results = np.zeros(((rx_len_samples-len(tx_sig))+1, np.shape(rx_sig_reshaped)[1]//n_stack), dtype=np.csingle)
print(f"shape of xcorr_results: {np.shape(xcorr_results)}")

for res_idx in range(np.shape(xcorr_results)[1]):
    stacked = np.sum(rx_sig_reshaped[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
    xcorr_results[:, res_idx] = sp.correlate(stacked, tx_sig, mode='valid', method='auto')

In [ ]:
fig, ax = plt.subplots(figsize=(10,6), facecolor='white')
slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(xcorr_results)[1])
distance_to_reflector = np.linspace(0, np.shape(xcorr_results)[0]/sample_rate, np.shape(xcorr_results)[0]) * sig_speed / 2
zero_sample_idx = 159
distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx]

p = ax.pcolormesh(slow_time, distance_to_reflector, 20*np.log10(np.abs(xcorr_results)), shading='auto')
clb = fig.colorbar(p)
clb.set_label('Power [dB]')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Relative distance to target [m]')
ax.set_ylim(0,1000)
ax.set_ylim(0,100)
plt.show()

In [ ]:
# Must enable widgets (see imports block) for this to work

# def plot_timestep(idx):
#     fig, ax = plt.subplots()
#     ax.plot(distance_to_reflector, 20*np.log10(np.abs(xcorr_results[:,idx])))
#     ax.set_ylim(0,100)
#     ax.set_xlim(0, 100)
#     ax.grid()
#     return fig, ax
    
# widgets.interact(plot_timestep, idx=widgets.IntSlider(min=0, max=np.shape(xcorr_results)[1]-1, step=1, value=0))